In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

In [ ]:
AA_TOKEN=os.getenv("AA_TOKEN")
NAMESPACE=os.getenv("AA_NAMESPACE")

# Document Index

First, let's have a look at the UI.  
Visit: https://app.document-index.aleph-alpha.com

## Collection

### Creating A Collection

In [2]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
)
def create_collection(collection_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - use DocumentIndexClient to create a collection
    pass

CUSTOM_PREFIX = "your-name" # TODO: add your name here, so that your collection names don't collide with the others.

COLLECTION_NAME=f"{CUSTOM_PREFIX}-demo"
COLLECTION_NAME_DELETE=f"{CUSTOM_PREFIX}-demo-collection-delete"

create_collection(COLLECTION_NAME)
create_collection(COLLECTION_NAME_DELETE)

### Get all the Collections

In [ ]:
def get_collections():
    # TODO
    #   - create DocumentIndexClient object
    #   - use DocumentIndexClient to get the collections
    #   - return the collections
    collections = []
    return collections
collections = get_collections()
collections

### Delete a Collection

In [ ]:
def _delete_collection(collection_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - use DocumentIndexClient object to delete the collection given by the CollectionPath
    pass

_delete_collection(COLLECTION_NAME_DELETE)
# get collections to see if it was successfull
get_collections()


## Documents

### Parse Content of Document (PDF, TXT and DOCX) 

In [ ]:
from parser import FileParser
import os

def get_file_content(path) -> tuple[str, str]:
    """Returns content and file name."""
    parser = FileParser()
    file_content, file_name = parser.parse(path)
    return file_content, file_name


PATH_PDF = os.path.join("..", "..", "example_documents", "Der-Schattenmann.pdf")
PATH_TXT = os.path.join("..", "..", "example_documents", "Plyscraper.txt")
PATH_DOCX = os.path.join("..", "..", "example_documents", "Erfundene-Programmiersprache.docx")


file_content, file_name = get_file_content(PATH_DOCX)

print("File name:", file_name, "\n", "-"*50)
print(file_content)


### Upload Content

In [ ]:

from intelligence_layer.connectors import (
    CollectionPath,
    DocumentContents,
    DocumentIndexClient,
    DocumentPath,
)
from datetime import datetime

def upload_document(file_path: str, collection_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - parse content of the file
    #   - create DocumentPath object
    #   - create DocumentContents object using the from_text class method
    #   - add metadata to the DocumentContents object
    #   - use DocumentIndexClient to add the document (DocumentPath, content)
    pass

upload_document(PATH_PDF, COLLECTION_NAME)
upload_document(PATH_TXT, COLLECTION_NAME)
upload_document(PATH_DOCX, COLLECTION_NAME)

### Get all Documents within a Collection

In [ ]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
)
def get_documents(collection_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - get documents using the DocumentIndexClient 
    #   - return documents
    documents = []
    return documents

documents = get_documents(COLLECTION_NAME)

for document in documents:
    print(document.model_dump_json(indent=2))

### Get Single Document (and Content)

In [ ]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
    DocumentPath
)
def get_document(collection_name: str, document_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - create DocumentPath object
    #   - use DocumentIndexClient to get a single document given by DocumentPath
    document = None
    return document

document_name_pdf = "Der-Schattenmann.pdf"
document_name_txt = "Plyscraper.txt"
document_name_docx = "Erfundene-Programmiersprache.docx"

document = get_document(
    collection_name=COLLECTION_NAME,
    document_name=document_name_pdf
)

print(document.model_dump_json(indent=2))

## Data Retriever

### Create Index

In [ ]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
    IndexConfiguration,
    IndexPath,
)

def _create_index(index_name: str, chunk_size: int):
    # TODO
    #   - create DocumentIndexClient object
    #   - create IndexPath object
    #   - create IndexConfiguration object
    #   - use DocumentIndexClient to create a index (IndexPath, IndexConfiguration)
    return None

INDEX_NAME = f"{CUSTOM_PREFIX}demo-index"
chunk_size = 128
_create_index(INDEX_NAME, chunk_size)

### Assign Index to Collection

In [ ]:
def _assign_index_to_collection(collection_name: str, index_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - use DocumentIndexClient to assign the index to the collection
    return None

_assign_index_to_collection(COLLECTION_NAME, INDEX_NAME)

### Get all Indexes

In [ ]:
def get_all_indexes(collection_name: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create CollectionPath object
    #   - use DocumentIndexClient to list the assined indexes
    indexes = []
    return indexes

indexes = get_all_indexes(collection_name=COLLECTION_NAME)
indexes


### Similarity Search

In [ ]:
from intelligence_layer.connectors import (
    CollectionPath,
    DocumentIndexClient,
    DocumentIndexRetriever,
)
def similariy_search(collection_name:str, index_name: str, query: str):
    # TODO
    #   - create DocumentIndexClient object
    #   - create DocumentIndexRetriever object
    #   - use DocumentIndexRetriever to get relevant documents for given query
    relevant_documents = []
    return relevant_documents

query = "plyscraper"
relevant_documents = similariy_search(
    collection_name=COLLECTION_NAME,
    index_name=INDEX_NAME,
    query=query
)
relevant_documents

# Upload Data for RAG

## Upload Custom Dataset

In [ ]:
ACCEPTED_FILE_EXTENSTIONS = ["txt", "docx", "pdf"]

In [26]:
collection_name = f"{CUSTOM_PREFIX}-rag-collection"
index_name = f"{CUSTOM_PREFIX}-rag-index"
chunk_size = 128
file_directory_path = os.path.abspath(os.path.join("..", "..", "example_documents"))

# TODO create a collection
for root, dirs, files in os.walk(file_directory_path):
    for file in files:
        file_extenstion = file[file.rfind(".")+1:]
        if file_extenstion in ACCEPTED_FILE_EXTENSTIONS:
            file_path = os.path.join(root,file)
            # TODO upload document

# TODO create index
# TODO assign index to collection
# TODO get documents (to check if it worked)

## Upload quad dataset

1. Load the huggingface dataset located at `deepset/germanquad` and sample a subset (n=10).
2. Create an index and assign it to the collection.
3. Load the content of the `context` columns as documents into collections.

In [23]:
from datasets import load_dataset

collection_name = f"{CUSTOM_PREFIX}-rag-collection"
index_name = f"{CUSTOM_PREFIX}-rag-index"

HF_DATASET_NAME = "deepset/germanquad"